In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 데이터 저장용 리스트
data = []

# 요청 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# 페이지 번호 순차적으로 접근
file_name = '에이텍모빌1.xlsx'
for page in range(1, 6):  # 1부터 5페이지까지
    url = f"https://finance.naver.com/item/frgn.naver?code=224110&page={page}"
    
    response = requests.get(url, headers=headers)
    
    # 상태 코드 확인
    if response.status_code == 200:
        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 'type2' 테이블에서 모든 행 가져오기
        rows = soup.select("table.type2 tr")
        
        for row in rows:
            cols = row.select("td")  # 열(td) 태그 선택
            
            # 필요한 데이터가 포함된 행인지 확인
            if len(cols) >= 9:  # 최소 9개의 열이 있어야 모든 데이터 추출 가능
                date = cols[0].text.strip()  # 날짜
                close_price = cols[1].text.strip()  # 종가
                change_amount = cols[2].text.strip()  # 전일 대비
                change_rate = cols[3].text.strip().replace('%','')  # 변동률
                volume = cols[4].text.strip()  # 거래량
                foreign_net = cols[5].text.strip()  # 외국인 순매매
                foreign_cum = cols[6].text.strip()  # 외국인 누적
                institution_net = cols[7].text.strip()  # 기관 순매매
                foreign_rate = cols[8].text.strip()  # 외국인 지분율
                
                # 데이터 저장
                data.append([date, close_price, change_amount, change_rate, volume, foreign_net, foreign_cum, institution_net, foreign_rate])
    else:
        print(f"페이지 {page}를 불러오지 못했습니다. 상태 코드:", response.status_code)

# 데이터 역순으로 변경
data.reverse()  # <- 이 부분 추가

# 데이터프레임으로 저장 (선택 사항)
columns = ['날짜', '종가', '전일 대비', '변동률', '거래량', '외국인 순매매', '외국인 누적', '기관 순매매', '외국인 지분율']
df = pd.DataFrame(data, columns=columns)

# 데이터프레임 확인
print(df)

# 엑셀 파일로 저장
df.to_excel(file_name, index=False)
print(f"모든 데이터가 '{file_name}'로 저장되었습니다.")


            날짜      종가            전일 대비    변동률      거래량 외국인 순매매   외국인 누적  \
0   2024.08.27  13,930  하락\n\t\t\t\t190  -1.35   25,710  +5,000   -3,033   
1   2024.08.28  13,960   상승\n\t\t\t\t30  +0.22   13,053  +5,000   -2,699   
2   2024.08.29  13,940   하락\n\t\t\t\t20  -0.14   16,174  +5,391   -6,724   
3   2024.08.30  13,870   하락\n\t\t\t\t70  -0.50   25,045  +9,927   -1,672   
4   2024.09.02  13,800   하락\n\t\t\t\t70  -0.50   17,870  +6,884   -6,049   
..         ...     ...              ...    ...      ...     ...      ...   
95  2025.01.20  17,190  하락\n\t\t\t\t410  -2.33  103,996    -233   -1,567   
96  2025.01.21  16,610  하락\n\t\t\t\t580  -3.37   98,745      -7  +16,838   
97  2025.01.22  16,290  하락\n\t\t\t\t320  -1.93   60,789    +220   -1,840   
98  2025.01.23  16,500  상승\n\t\t\t\t210  +1.29   47,164    -219     +108   
99  2025.01.24  17,400  상승\n\t\t\t\t900  +5.45  180,229      -1  -16,634   

     기관 순매매 외국인 지분율  
0   349,483   6.54%  
1   346,784   6.49%  
2   340,060   6.37%  